<li>TF (term frequency) : number of time a word occurs in a document</li>
<li>IDF (inverse document frequency) : log fraction of the total number of documents by the number of documents containing the term </li>

<i>We first split all the congress members into two groups : democratics and republicans </i>

In [1]:
import pandas as pd


congress1 = pd.read_csv('../data/H113.csv')
congress2 = pd.read_csv('../data/H114.csv')
congress3 = pd.read_csv('../data/H115.csv')

congress_members = pd.concat([congress1,congress2, congress3])
congress_members = congress_members.drop_duplicates()

republicans, democratics = [], []

for congressman in congress1.WikiPageName.sort_values():
    party = congress1[congress1.WikiPageName == congressman].Party.values
    if (str(party[0])) == "Republican":
        republicans.append(congressman)
    else:
        democratics.append(congressman)
   
republicans_new, democratics_new = [], []
for elt in republicans:
    for el in elt.split("_"):
        republicans_new.append(el.replace("(","").replace(")",""))
        
for elt in democratics:
    for el in elt.split("_"):
        democratics_new.append(el.replace("(","").replace(")",""))

republicans_new = list(set(republicans_new))# list of all names and surnames of politicians
democratics_new = list(set(democratics_new))

congress_men = set(republicans_new + democratics_new)

FileNotFoundError: File b'../data/H113.csv' does not exist

In [ ]:
import nltk
import re
import requests
import json
from nltk.corpus import stopwords
from nltk import word_tokenize


baseurl = "http://en.wikipedia.org/w/api.php/?"
action = "action=query"
content = "prop=extracts&exlimit=max&explaintext"
rvprop ="rvprop=timestamp|content"
dataformat = "format=json"
rvdir = "rvdir=older" #sort revisions from newest to oldest
start = "rvend=2017-01-03T00:00:00Z" #start of my time period
end = "rvstart=2018-09-25T00:00:00Z" #end of my time period
limit = "rvlimit=1" #consider only the first revision

file = open("../data/republicans.txt","w")
republicans_words_list = []
for member in republicans:
    title = "titles=" + member
    query = "%s%s&%s&%s&%s&%s&%s&%s&%s&%s" % (baseurl, action, title, content, rvprop, dataformat, rvdir, end, start, limit)
    text = requests.get(query).text
    text = str(json.loads(text)["query"]["pages"])
    text = text.strip()
    tokens = word_tokenize(text)
    
    tokens = [w for w in tokens if re.search(r'\w',w)]
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    tokens = [word for word in tokens if not re.search(r'[0-9]+$',word)]
    tokens = [re.sub('[^A-Za-z0-9]+', '', w) for w in tokens]
    
    tokens = [w.lower() for w in tokens if w not in congress_men]
    file.write(" ".join(tokens))
    print(republicans.index(member))

file.close()


In [2]:
baseurl = "http://en.wikipedia.org/w/api.php/?"
action = "action=query"
content = "prop=extracts&exlimit=max&explaintext"
rvprop ="rvprop=timestamp|content"
dataformat = "format=json"
rvdir = "rvdir=older" #sort revisions from newest to oldest
start = "rvend=2017-01-03T00:00:00Z" #start of my time period
end = "rvstart=2018-09-25T00:00:00Z" #end of my time period
limit = "rvlimit=1" #consider only the first revision

file = open("../data/democratics.txt","w")
democratics_words_list = []
for member in democratics:
    title = "titles=" + member
    query = "%s%s&%s&%s&%s&%s&%s&%s&%s&%s" % (baseurl, action, title, content, rvprop, dataformat, rvdir, end, start, limit)
    text = requests.get(query).text
    text = str(json.loads(text)["query"]["pages"])
    text = text.strip()
    tokens = word_tokenize(text)
    
    tokens = [w for w in tokens if re.search(r'\w',w)]
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    tokens = [word for word in tokens if not re.search(r'[0-9]+$',word)]
    tokens = [re.sub('[^A-Za-z0-9]+', '', w) for w in tokens]
    
    tokens = [w.lower() for w in tokens if w not in congress_men]
    file.write(" ".join(tokens))
    print(democratics.index(member))

file.close()

NameError: name 'requests' is not defined

In [ ]:
import nltk

dem = open("../data/democratics.txt","r").read()
dem_words = word_tokenize(dem)
dem_text = nltk.Text(dem_words)

fd_dem = FreqDist(dem_text)
print(fd_dem.most_common(30))

TFs are not necessarily a good description because it can give importance to meaningless words like "the". 
That is why IDF is a good tool because it gives less importance to word that occurs too many times.

In [ ]:
from math import log

for elt in (fd_dem.most_common()[-5:]):
    print(elt[0] + " : " + str(log( len(dem_words) / int(elt[1]))))

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

wordcloud = WordCloud().generate(dem)
plt.imshow(wordcloud, interpolation='bilinear');
plt.axis("off");

<u>Modularity :</u> it mesures the strength of a network into modules. Networks with light modularity have dense connections between the nodes within modules but sparse between nodes of different modules.

In [ ]:
import pandas as pd
import networkx as nx
import re
import io

congress1 = pd.read_csv('../data/H115.csv')
path_folder = "../data/congress115/congress115"

congress113_graph = nx.Graph()
for congressman in congress1.WikiPageName.sort_values():
    #new node with congressman's info
    congress113_graph.add_node(congressman,
                           state=congress1[congress1.WikiPageName == congressman].State,
                           party=congress1[congress1.WikiPageName == congressman].Party)
    
    f = io.open(path_folder+congressman+'.txt', 'r', encoding='utf-8').read()
    links = re.findall("\[\[(.*?)\]\]", f)
    for l in links:
        match = re.search(r'([a-zA-Z0-9_\s\(\)\-\,.]*)\|([a-zA-Z0-9_\s\(\)\-\,.]*)', l)
        if match == None:
            m = l.replace(" ", "_")
            if m in set(congress1.WikiPageName):
                congress113_graph.add_edge(congressman, m)
        else:
            m1 = match.group(1).replace(" ", "_")
            m2 = match.group(2).replace(" ", "_")
            if m1 in set(congress1.WikiPageName):
                congress113_graph.add_edge(congressman, m1)
            if m2 in set(congress1.WikiPageName):
                congress113_graph.add_edge(congressman, m2)

In [ ]:
L = len(congress113_graph.edges()) #total number of edges

list_attributes = nx.get_node_attributes(congress113_graph, 'party')

L_rep, L_dem = 0, 0 
for edge in congress113_graph.edges():
    if congress1[congress1.WikiPageName == edge[0]].Party.values[0] == congress1[congress1.WikiPageName == edge[1]].Party.values[0] == 'Republican':
        L_rep += 1
    elif congress1[congress1.WikiPageName == edge[0]].Party.values[0] == congress1[congress1.WikiPageName == edge[1]].Party.values[0] == 'Democratic':
        L_dem += 1
    #     print(list_attributes[edge[0]])

K_rep, K_dem = 0, 0
for elt in list(congress113_graph.degree()):
    if congress1[congress1.WikiPageName == elt[0]].Party.values[0] == 'Republican':
        K_rep += elt[1]
    else:
        K_dem += elt[1]

In [ ]:
modularity = (L_rep/L - pow(K_rep/(2*L),2)) + (L_dem/L - pow(K_dem/(2*L),2))
print(modularity)

The modularity is quiet low which means that nodes from different modules are well connected.

In [2]:
# using Louvain communities detection
import community

partition = community.best_partition(congress113_graph)

#drawing
size = float(len(set(partition.values())))
pos = nx.spring_layout(congress113_graph)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(congress113_graph, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(congress113_graph,pos, alpha=0.5)
plt.show()

ModuleNotFoundError: No module named 'community'